In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir("/content/drive/MyDrive/Courses/CSCI544-ANLP/CSCI544-Project/Shashank/siamese-BERT-fake-news-detection-LIAR/")

In [3]:
import pandas as pd
import numpy as np

## Loading LIAR Dataset

In [4]:

train_path = './LIAR-PLUS/dataset/train2.tsv'
test_path = './LIAR-PLUS/dataset/test2.tsv'
val_path = './LIAR-PLUS/dataset/val2.tsv'


In [5]:
train_df = pd.read_csv(train_path, sep="\t", header=None)
test_df = pd.read_csv(test_path, sep="\t", header=None)
val_df = pd.read_csv(val_path, sep="\t", header=None)

In [6]:
train_df = train_df.fillna(0)
test_df = test_df.fillna(0)
val_df = val_df.fillna(0)

In [7]:
# Define content and metadata components
renamed_columns = {
    0: "index",
    1: "statement_id",
    2: "label",
    3: "statement",
    4: "subject",
    5: "speaker",
    6: "speaker_job_title",
    7: "state_info",
    8: "party_affiliation",
    9: "barely_true_counts",
    10: "false_counts",
    11: "half_true_counts",
    12: "mostly_true_counts",
    13: "pants_on_fire_counts",
    14: "context",
    15: "justification"
}
train_df = train_df.rename(columns=renamed_columns)
val_df = val_df.rename(columns=renamed_columns)
test_df = test_df.rename(columns=renamed_columns)

In [8]:

# label_map = {
#     "true": 0,
#     "mostly-true": 0,
#     "half-true": 0,
#     "barely-true": 1,
#     "false": 1,
#     "pants-fire": 1
# }

label_map = {
    "true": 1,
    "mostly-true": 1,
    "half-true": 1,
    "barely-true": 0,
    "false": 0,
    "pants-fire": 0
}
to_binary_label = lambda label: label_map.get(label, 0)

In [9]:
y_train = train_df['label'].apply(to_binary_label).values
y_val = val_df['label'].apply(to_binary_label).values
y_test = test_df['label'].apply(to_binary_label).values

In [ ]:
X_train_statement = train_df['statement'].values
X_val_statement = val_df['statement'].values
X_test_statement = test_df['statement'].values

## Loading Bert Model

In [13]:
# !pip install pytorch-pretrained-bert transformers
! pip install transformers sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 101.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 79.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 26.4 MB/s eta 0:00:00


In [14]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
import time
from tqdm import tqdm
from transformers import RobertaModel, RobertaTokenizer
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
import copy


In [15]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
bert_model = RobertaModel.from_pretrained('roberta-base')
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

for param in bert_model.parameters():
    param.requires_grad = False

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
bert_model.to(device)

RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(50265, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0-11): 12 x RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropou

## Types of embeddings stored:
 - Statement CLS token only
 - Statement avg tokens

### Trian

In [ ]:
statements_with_CLS_token = []
statements_with_avg_token = []

In [ ]:
with torch.no_grad():
  for statement in tqdm(X_train_statement):
    encoded = tokenizer(statement, truncation=True, return_tensors='pt', add_special_tokens=True)
    input_ids = encoded.input_ids.to(device)
    attention_mask = encoded.attention_mask.to(device)
    
    embedding = bert_model(input_ids, attention_mask=attention_mask)
    embedding_CLS = embedding.last_hidden_state[:, 0, :] # CLS Embedding
    embedding_avg = torch.mean(embedding.last_hidden_state, dim=1) # Mean embedding
    
    statements_with_CLS_token.append(embedding_CLS.cpu())
    statements_with_avg_token.append(embedding_avg.cpu())

100%|██████████| 10240/10240 [03:46<00:00, 45.29it/s]


In [ ]:
statements_with_CLS_token = torch.cat(statements_with_CLS_token, dim=0)
statements_with_avg_token = torch.cat(statements_with_avg_token, dim=0)

In [ ]:
np.save('liar-rb-cls-stmt-train.npy', statements_with_CLS_token.numpy())
np.save('liar-rb-avg-stmt-train.npy', statements_with_avg_token.numpy())

### Val

In [ ]:
statements_with_CLS_token = []
statements_with_avg_token = []

In [ ]:
with torch.no_grad():
  for statement in tqdm(X_val_statement):
    encoded = tokenizer(statement, truncation=True, return_tensors='pt', add_special_tokens=True)
    input_ids = encoded.input_ids.to(device)
    attention_mask = encoded.attention_mask.to(device)
    
    embedding = bert_model(input_ids, attention_mask=attention_mask)
    embedding_CLS = embedding.last_hidden_state[:, 0, :] # CLS Embedding
    embedding_avg = torch.mean(embedding.last_hidden_state, dim=1) # Mean embedding
    
    statements_with_CLS_token.append(embedding_CLS.cpu())
    statements_with_avg_token.append(embedding_avg.cpu())

100%|██████████| 1284/1284 [00:26<00:00, 47.86it/s]


In [ ]:
statements_with_CLS_token = torch.cat(statements_with_CLS_token, dim=0)
statements_with_avg_token = torch.cat(statements_with_avg_token, dim=0)

In [ ]:
np.save('liar-rb-cls-stmt-val.npy', statements_with_CLS_token.numpy())
np.save('liar-rb-avg-stmt-val.npy', statements_with_avg_token.numpy())

### Test:

In [ ]:
statements_with_CLS_token = []
statements_with_avg_token = []

In [ ]:
with torch.no_grad():
  for statement in tqdm(X_test_statement):
    encoded = tokenizer(statement, truncation=True, return_tensors='pt', add_special_tokens=True)
    input_ids = encoded.input_ids.to(device)
    attention_mask = encoded.attention_mask.to(device)
    
    embedding = bert_model(input_ids, attention_mask=attention_mask)
    embedding_CLS = embedding.last_hidden_state[:, 0, :] # CLS Embedding
    embedding_avg = torch.mean(embedding.last_hidden_state, dim=1) # Mean embedding
    
    statements_with_CLS_token.append(embedding_CLS.cpu())
    statements_with_avg_token.append(embedding_avg.cpu())

100%|██████████| 1267/1267 [00:20<00:00, 61.07it/s]


In [ ]:
statements_with_CLS_token = torch.cat(statements_with_CLS_token, dim=0)
statements_with_avg_token = torch.cat(statements_with_avg_token, dim=0)

In [ ]:
np.save('liar-rb-cls-stmt-test.npy', statements_with_CLS_token.numpy())
np.save('liar-rb-avg-stmt-test.npy', statements_with_avg_token.numpy())

In [ ]:
del statements_with_CLS_token
del statements_with_avg_token

## Classification

In [ ]:
def get_best_model(model, true, pred, model_name):
  torch.save(model.state_dict(), model_name)
  return {
      "acc": accuracy_score(true, pred),
      "auc": roc_auc_score(true, pred),
      "f1": f1_score(true, pred),
  }

### CLS embedding

In [ ]:
train_embeddings = torch.from_numpy(np.load('liar-rb-cls-stmt-train.npy'))
val_embeddings = torch.from_numpy(np.load('liar-rb-cls-stmt-val.npy'))
test_embeddings = torch.from_numpy(np.load('liar-rb-cls-stmt-test.npy'))


In [ ]:
train_dataset = torch.utils.data.TensorDataset(
    train_embeddings,
    torch.tensor(y_train)
)

val_dataset = torch.utils.data.TensorDataset(
    val_embeddings,
    torch.tensor(y_val)
)

test_dataset = torch.utils.data.TensorDataset(
    test_embeddings,
    torch.tensor(y_test)
)

In [ ]:
batch_size = 256
train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle = False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
classifier = torch.nn.Sequential(
    torch.nn.Linear(768, 256),
    torch.nn.ReLU(),
    torch.nn.Linear(256, 2)
)
classifier

Sequential(
  (0): Linear(in_features=768, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=2, bias=True)
)

In [ ]:
optimizer = torch.optim.Adam(classifier.parameters(), lr=1e-4)
epochs = 200
classifier = classifier.to(device)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
best_val_f1 = 0
best_test_f1 = 0
best_val_auc = 0
best_test_auc = 0
best_val_acc = 0
best_test_acc = 0

f1_val_model = {}
f1_test_model = {}

acc_val_model = {}
acc_test_model = {}

auc_val_model = {}
auc_test_model = {}

for epoch in range(epochs):

  print(f"Epoch {epoch + 1}/ {epochs}")
  print("-" * 100)
  # Train loop
  classifier.train()
  total_train_loss = 0

  for inputs, labels in tqdm(train_loader):
    inputs = inputs.to(device)
    labels = labels.to(device)
    logits = classifier(inputs)
    current_loss = loss_fn(logits, labels)
    total_train_loss += current_loss.item()
    classifier.zero_grad()
    current_loss.backward()
    optimizer.step()

  avg_train_loss = total_train_loss / len(train_loader)

  classifier.eval()
  total_val_loss = 0
  pred = torch.tensor([])
  true = torch.tensor([])
  for inputs, labels in tqdm(val_loader):
    inputs = inputs.to(device)
    labels = labels.to(device)
    logits = classifier(inputs)
    current_loss = loss_fn(logits, labels)
    total_val_loss += current_loss.item()
    pred = torch.cat([pred, torch.argmax(logits, dim=1).cpu()], dim=0)
    true = torch.cat([true, labels.cpu()], dim=0)
  avg_val_loss = total_val_loss / len(val_loader)
  val_acc = accuracy_score(true, pred)
  val_auc = roc_auc_score(true, pred)
  val_f1 = f1_score(true, pred)
  if val_f1 > best_val_f1:
    f1_val_model = get_best_model(classifier, true, pred, 'liar-rb-stmt-cls-bvalf1.pt')
    best_val_f1 = val_f1

  if val_acc > best_val_acc:
    acc_val_model = get_best_model(classifier, true, pred, 'liar-rb-stmt-cls-bvalacc.pt')
    best_val_acc = val_acc

  if val_auc > best_val_auc:
    auc_val_model = get_best_model(classifier, true, pred, 'liar-rb-stmt-cls-bvalauc.pt')
    best_val_auc = val_auc

  classifier.eval()
  total_test_loss = 0
  pred = torch.tensor([])
  true = torch.tensor([])
  for inputs, labels in tqdm(test_loader):
    inputs = inputs.to(device)
    labels = labels.to(device)
    logits = classifier(inputs)
    current_loss = loss_fn(logits, labels)
    total_test_loss += current_loss.item()
    pred = torch.cat([pred, torch.argmax(logits, dim=1).cpu()], dim=0)
    true = torch.cat([true, labels.cpu()], dim=0)
  avg_test_loss = total_test_loss / len(test_loader)
  test_acc = accuracy_score(true, pred)
  test_f1 = f1_score(true, pred)
  test_auc = roc_auc_score(true, pred)
  if test_f1 > best_test_f1:
    f1_test_model = get_best_model(classifier, true, pred, 'liar-rb-stmt-cls-btestf1.pt')
    best_test_f1 = test_f1

  if test_acc > best_test_acc:
    acc_test_model = get_best_model(classifier, true, pred, 'liar-rb-stmt-cls-btestacc.pt')
    best_test_acc = test_acc

  if test_auc > best_test_auc:
    auc_test_model = get_best_model(classifier, true, pred, 'liar-rb-stmt-cls-btestauc.pt')
    best_test_auc = test_auc

  print(f"Train loss: {avg_train_loss}, Val loss: {avg_val_loss}, Test loss: {avg_test_loss}")
  print(f"Val acc: {val_acc}, Test acc: {test_acc}")
  print(f"Val f1: {val_f1}, Test f1: {test_f1}")


Epoch 1/ 200
----------------------------------------------------------------------------------------------------


100%|██████████| 5/5 [00:00<00:00, 363.43it/s]


Train loss: 0.47359714433550837, Val loss: 0.6460243761539459, Test loss: 0.7093613266944885
Val acc: 0.6440809968847352, Test acc: 0.6093133385951065
Val f1: 0.6251025430680887, Test f1: 0.5568487018800359
Epoch 2/ 200
----------------------------------------------------------------------------------------------------


100%|██████████| 5/5 [00:00<00:00, 361.42it/s]


Train loss: 0.4729935094714165, Val loss: 0.6551685233910879, Test loss: 0.7070224046707153
Val acc: 0.6495327102803738, Test acc: 0.6203630623520127
Val f1: 0.6024734982332155, Test f1: 0.5316455696202531
Epoch 3/ 200
----------------------------------------------------------------------------------------------------


...

Train loss: 0.42889572232961654, Val loss: 0.6806646386782328, Test loss: 0.7314980030059814
Val acc: 0.6495327102803738, Test acc: 0.6179952644041041
Val f1: 0.6073298429319371, Test f1: 0.5372848948374761
Epoch 199/ 200
----------------------------------------------------------------------------------------------------


100%|██████████| 5/5 [00:00<00:00, 224.73it/s]


Train loss: 0.4288746900856495, Val loss: 0.6863555312156677, Test loss: 0.7335659980773925
Val acc: 0.6479750778816199, Test acc: 0.6148382004735596
Val f1: 0.5942549371633753, Test f1: 0.514910536779324
Epoch 200/ 200
----------------------------------------------------------------------------------------------------


100%|██████████| 5/5 [00:00<00:00, 228.48it/s]

Train loss: 0.42840354442596434, Val loss: 0.6921965976556143, Test loss: 0.7357542514801025
Val acc: 0.6448598130841121, Test acc: 0.6172059984214681
Val f1: 0.5839416058394159, Test f1: 0.5086119554204661


In [ ]:
print("-" * 100)
print("Using CLS token Embedding")
print("-" * 100)
print("Best Validation Models")
print(f"F1 score: {f1_val_model}")
print(f"Acc score: {acc_val_model}")
print(f"AUC score: {auc_val_model}")
print("-" * 100)
print("Best Test Models")
print(f"F1 score: {f1_test_model}")
print(f"Acc score: {acc_test_model}")
print(f"AUC score: {auc_test_model}")

----------------------------------------------------------------------------------------------------
Using CLS token Embedding
----------------------------------------------------------------------------------------------------
Best Validation Models
F1 score: {'acc': 0.6386292834890965, 'auc': 0.638730655571973, 'f1': 0.6299840510366826}
Acc score: {'acc': 0.6557632398753894, 'auc': 0.6512170464266273, 'f1': 0.6003616636528029}
AUC score: {'acc': 0.6549844236760125, 'auc': 0.6512267672447314, 'f1': 0.6083112290008841}
----------------------------------------------------------------------------------------------------
Best Test Models
F1 score: {'acc': 0.6085240726124704, 'auc': 0.6080115590540014, 'f1': 0.5738831615120276}
Acc score: {'acc': 0.6235201262825573, 'auc': 0.6035794064461226, 'f1': 0.5087538619979403}
AUC score: {'acc': 0.6219415943172849, 'auc': 0.6115572811403042, 'f1': 0.5502347417840376}


### AVG embedding

In [ ]:
train_embeddings = torch.from_numpy(np.load('liar-rb-avg-stmt-train.npy'))
val_embeddings = torch.from_numpy(np.load('liar-rb-avg-stmt-val.npy'))
test_embeddings = torch.from_numpy(np.load('liar-rb-avg-stmt-test.npy'))


In [ ]:
train_dataset = torch.utils.data.TensorDataset(
    train_embeddings,
    torch.tensor(y_train)
)

val_dataset = torch.utils.data.TensorDataset(
    val_embeddings,
    torch.tensor(y_val)
)

test_dataset = torch.utils.data.TensorDataset(
    test_embeddings,
    torch.tensor(y_test)
)

In [ ]:
batch_size = 256
train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle = False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
classifier = torch.nn.Sequential(
    torch.nn.Linear(768, 256),
    torch.nn.ReLU(),
    torch.nn.Linear(256, 2)
)
classifier

Sequential(
  (0): Linear(in_features=768, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=2, bias=True)
)

In [ ]:
optimizer = torch.optim.Adam(classifier.parameters(), lr=1e-4)
epochs = 200
classifier = classifier.to(device)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
best_val_f1 = 0
best_test_f1 = 0
best_val_auc = 0
best_test_auc = 0
best_val_acc = 0
best_test_acc = 0

f1_val_model = {}
f1_test_model = {}

acc_val_model = {}
acc_test_model = {}

auc_val_model = {}
auc_test_model = {}

for epoch in range(epochs):

  print(f"Epoch {epoch + 1}/ {epochs}")
  print("-" * 100)
  # Train loop
  classifier.train()
  total_train_loss = 0

  for inputs, labels in tqdm(train_loader):
    inputs = inputs.to(device)
    labels = labels.to(device)
    logits = classifier(inputs)
    current_loss = loss_fn(logits, labels)
    total_train_loss += current_loss.item()
    classifier.zero_grad()
    current_loss.backward()
    optimizer.step()

  avg_train_loss = total_train_loss / len(train_loader)

  classifier.eval()
  total_val_loss = 0
  pred = torch.tensor([])
  true = torch.tensor([])
  for inputs, labels in tqdm(val_loader):
    inputs = inputs.to(device)
    labels = labels.to(device)
    logits = classifier(inputs)
    current_loss = loss_fn(logits, labels)
    total_val_loss += current_loss.item()
    pred = torch.cat([pred, torch.argmax(logits, dim=1).cpu()], dim=0)
    true = torch.cat([true, labels.cpu()], dim=0)
  avg_val_loss = total_val_loss / len(val_loader)
  val_acc = accuracy_score(true, pred)
  val_auc = roc_auc_score(true, pred)
  val_f1 = f1_score(true, pred)
  if val_f1 > best_val_f1:
    f1_val_model = get_best_model(classifier, true, pred, 'liar-rb-stmt-avg-bvalf1.pt')
    best_val_f1 = val_f1

  if val_acc > best_val_acc:
    acc_val_model = get_best_model(classifier, true, pred, 'liar-rb-stmt-avg-bvalacc.pt')
    best_val_acc = val_acc

  if val_auc > best_val_auc:
    auc_val_model = get_best_model(classifier, true, pred, 'liar-rb-stmt-avg-bvalauc.pt')
    best_val_auc = val_auc

  classifier.eval()
  total_test_loss = 0
  pred = torch.tensor([])
  true = torch.tensor([])
  for inputs, labels in tqdm(test_loader):
    inputs = inputs.to(device)
    labels = labels.to(device)
    logits = classifier(inputs)
    current_loss = loss_fn(logits, labels)
    total_test_loss += current_loss.item()
    pred = torch.cat([pred, torch.argmax(logits, dim=1).cpu()], dim=0)
    true = torch.cat([true, labels.cpu()], dim=0)
  avg_test_loss = total_test_loss / len(test_loader)
  test_acc = accuracy_score(true, pred)
  test_f1 = f1_score(true, pred)
  test_auc = roc_auc_score(true, pred)
  if test_f1 > best_test_f1:
    f1_test_model = get_best_model(classifier, true, pred, 'liar-rb-stmt-avg-btestf1.pt')
    best_test_f1 = test_f1

  if test_acc > best_test_acc:
    acc_test_model = get_best_model(classifier, true, pred, 'liar-rb-stmt-avg-btestacc.pt')
    best_test_acc = test_acc

  if test_auc > best_test_auc:
    auc_test_model = get_best_model(classifier, true, pred, 'liar-rb-stmt-avg-btestauc.pt')
    best_test_auc = test_auc

  print(f"Train loss: {avg_train_loss}, Val loss: {avg_val_loss}, Test loss: {avg_test_loss}")
  print(f"Val acc: {val_acc}, Test acc: {test_acc}")
  print(f"Val f1: {val_f1}, Test f1: {test_f1}")


Epoch 1/ 200
----------------------------------------------------------------------------------------------------


100%|██████████| 5/5 [00:00<00:00, 337.66it/s]


Train loss: 0.6807685270905495, Val loss: 0.6841397682825724, Test loss: 0.6709428906440735
Val acc: 0.5264797507788161, Test acc: 0.5659037095501184
Val f1: 0.047021943573667714, Test f1: 0.038461538461538464
Epoch 2/ 200
----------------------------------------------------------------------------------------------------


100%|██████████| 5/5 [00:00<00:00, 373.60it/s]


Train loss: 0.6666918352246285, Val loss: 0.6734551588694254, Test loss: 0.6598305821418762
Val acc: 0.5802180685358256, Test acc: 0.5895816890292028
Val f1: 0.35449101796407184, Test f1: 0.32989690721649484
Epoch 3/ 200
----------------------------------------------------------------------------------------------------


...

Train loss: 0.4437213227152824, Val loss: 0.64266503850619, Test loss: 0.6797657966613769
Val acc: 0.6347352024922118, Test acc: 0.6337805840568271
Val f1: 0.5925282363162468, Test f1: 0.5564053537284895
Epoch 199/ 200
----------------------------------------------------------------------------------------------------


100%|██████████| 5/5 [00:00<00:00, 275.08it/s]


Train loss: 0.4427484184503555, Val loss: 0.6491038898626963, Test loss: 0.6818380713462829
Val acc: 0.6347352024922118, Test acc: 0.6282557221783741
Val f1: 0.5831111111111111, Test f1: 0.533201189296333
Epoch 200/ 200
----------------------------------------------------------------------------------------------------


100%|██████████| 5/5 [00:00<00:00, 284.25it/s]

Train loss: 0.44182414188981056, Val loss: 0.6450686951478323, Test loss: 0.682219123840332
Val acc: 0.632398753894081, Test acc: 0.6322020520915549
Val f1: 0.5979557069846678, Test f1: 0.5661080074487896


In [ ]:
print("-" * 100)
print("Using AVG token Embedding")
print("-" * 100)
print("Best Validation Models")
print(f"F1 score: {f1_val_model}")
print(f"Acc score: {acc_val_model}")
print(f"AUC score: {auc_val_model}")
print("-" * 100)
print("Best Test Models")
print(f"F1 score: {f1_test_model}")
print(f"Acc score: {acc_test_model}")
print(f"AUC score: {auc_test_model}")

----------------------------------------------------------------------------------------------------
Using AVG token Embedding
----------------------------------------------------------------------------------------------------
Best Validation Models
F1 score: {'acc': 0.6456386292834891, 'auc': 0.644772144023641, 'f1': 0.6279640228945216}
Acc score: {'acc': 0.6635514018691588, 'auc': 0.6599025974025974, 'f1': 0.619047619047619}
AUC score: {'acc': 0.6635514018691588, 'auc': 0.6599025974025974, 'f1': 0.619047619047619}
----------------------------------------------------------------------------------------------------
Best Test Models
F1 score: {'acc': 0.6369376479873717, 'auc': 0.6326100060277277, 'f1': 0.5900178253119429}
Acc score: {'acc': 0.6456195737963694, 'auc': 0.6217600964436407, 'f1': 0.5166846071044134}
AUC score: {'acc': 0.6369376479873717, 'auc': 0.6326100060277277, 'f1': 0.5900178253119429}


## LIAR Results

```
LIAR Results

Considering STATEMENT ONLY WITH ROBERTA-BASE
----------------------------------------------------------------------------------------------------
Using CLS token Embedding
----------------------------------------------------------------------------------------------------
Best Validation Models
F1 score: {'acc': 0.6386292834890965, 'auc': 0.638730655571973, 'f1': 0.6299840510366826}
Acc score: {'acc': 0.6557632398753894, 'auc': 0.6512170464266273, 'f1': 0.6003616636528029}
AUC score: {'acc': 0.6549844236760125, 'auc': 0.6512267672447314, 'f1': 0.6083112290008841}
----------------------------------------------------------------------------------------------------
Best Test Models
F1 score: {'acc': 0.6085240726124704, 'auc': 0.6080115590540014, 'f1': 0.5738831615120276}
Acc score: {'acc': 0.6235201262825573, 'auc': 0.6035794064461226, 'f1': 0.5087538619979403}
AUC score: {'acc': 0.6219415943172849, 'auc': 0.6115572811403042, 'f1': 0.5502347417840376}

----------------------------------------------------------------------------------------------------
Using AVG token Embedding
----------------------------------------------------------------------------------------------------
Best Validation Models
F1 score: {'acc': 0.6456386292834891, 'auc': 0.644772144023641, 'f1': 0.6279640228945216}
Acc score: {'acc': 0.6635514018691588, 'auc': 0.6599025974025974, 'f1': 0.619047619047619}
AUC score: {'acc': 0.6635514018691588, 'auc': 0.6599025974025974, 'f1': 0.619047619047619}
----------------------------------------------------------------------------------------------------
Best Test Models
F1 score: {'acc': 0.6369376479873717, 'auc': 0.6326100060277277, 'f1': 0.5900178253119429}
Acc score: {'acc': 0.6456195737963694, 'auc': 0.6217600964436407, 'f1': 0.5166846071044134}
AUC score: {'acc': 0.6369376479873717, 'auc': 0.6326100060277277, 'f1': 0.5900178253119429}
```




```
Taking this best model from LIAR (F1-score)

On Fakeddit:

acc: 0.39370225768120864
f1: 0.530724862372062
auc: 0.4751307836003426


On Fake v/s Real FNC
acc: 0.4925833333333333
f1: 0.6184884901191714
auc: 0.4915650056063706

On Clickbait v/s Real FNC
acc: 0.4448666666666667
f1: 0.5970676473434627
auc: 0.4436999572899229
```

## Loading Fakedit Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


```
! unzip "/content/drive/MyDrive/Courses/CSCI544-ANLP/Project/544 Project Michael/fakedditEMBED.zip" -d /content/drive/MyDrive/Courses/CSCI544-ANLP/Project/fakeddit_emb/
Archive:  /content/drive/MyDrive/Courses/CSCI544-ANLP/Project/544 Project Michael/fakedditEMBED.zip
  inflating: /content/drive/MyDrive/Courses/CSCI544-ANLP/Project/fakeddit_emb/train10bd.tsv  
  inflating: /content/drive/MyDrive/Courses/CSCI544-ANLP/Project/fakeddit_emb/valid2bd.tsv  
```

```
! unzip "/content/drive/MyDrive/Courses/CSCI544-ANLP/Project/544 Project Michael/FNC-csv.zip" -d /content/drive/MyDrive/Courses/CSCI544-ANLP/Project/FNC_csv/
Archive:  /content/drive/MyDrive/Courses/CSCI544-ANLP/Project/544 Project Michael/FNC-csv.zip
  inflating: /content/drive/MyDrive/Courses/CSCI544-ANLP/Project/FNC_csv/clickbait.csv  
  inflating: /content/drive/MyDrive/Courses/CSCI544-ANLP/Project/FNC_csv/fakers.csv  
  inflating: /content/drive/MyDrive/Courses/CSCI544-ANLP/Project/FNC_csv/reliable.csv
```

```
! unzip "/content/drive/MyDrive/Courses/CSCI544-ANLP/Project/544 Project Michael/FNCEMBED.zip" -d /content/drive/MyDrive/Courses/CSCI544-ANLP/Project/FNC_emb/

Archive:  /content/drive/MyDrive/Courses/CSCI544-ANLP/Project/544 Project Michael/FNCEMBED.zip
  inflating: /content/drive/MyDrive/Courses/CSCI544-ANLP/Project/FNC_emb/click-real-testsplit.tsv  
  inflating: /content/drive/MyDrive/Courses/CSCI544-ANLP/Project/FNC_emb/fake-real-testsplit.tsv  
  inflating: /content/drive/MyDrive/Courses/CSCI544-ANLP/Project/FNC_emb/fake-real-trainsplit.tsv  
```

In [ ]:

import os
os.chdir("/content/drive/MyDrive/Courses/CSCI544-ANLP/Project/siamese-BERT-fake-news-detection-LIAR/")

In [ ]:
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
import time
from tqdm import tqdm
# from transformers import RobertaModel, RobertaTokenizer
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
import copy
from torch import tensor


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:

train_path = '/content/drive/MyDrive/Courses/CSCI544-ANLP/Project/fakeddit_emb/train10bd.tsv'
val_path = '/content/drive/MyDrive/Courses/CSCI544-ANLP/Project/fakeddit_emb/valid2bd.tsv'


In [ ]:
# train_df = pd.read_csv(train_path, sep="\t")
val_df = pd.read_csv(val_path, sep="\t")

In [ ]:
val_df = val_df.loc[(val_df.score > 0)]
# train_df = train_df.loc[(train_df.score > 0)]

In [ ]:
val_df['embedding'] = val_df['embedding'].apply(lambda x: eval(x))

In [ ]:
val_embeddings = torch.cat(list(val_df.embedding.values), dim=0)

In [ ]:
val_embeddings.shape

torch.Size([58910, 768])

In [ ]:
y_val = val_df['2-way-label'].values
y_val

array([1, 1, 0, ..., 0, 0, 0])

In [ ]:
classifier = torch.nn.Sequential(
    torch.nn.Linear(768, 256),
    torch.nn.ReLU(),
    torch.nn.Linear(256, 2)
)
classifier.load_state_dict(torch.load('liar-rb-stmt-avg-bvalf1.pt'))
classifier = classifier.to(device)
classifier

Sequential(
  (0): Linear(in_features=768, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=2, bias=True)
)

In [ ]:
val_dataset = torch.utils.data.TensorDataset(
    val_embeddings,
    torch.tensor(y_val)
)
batch_size = 256
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle = False)


In [ ]:
pred = torch.tensor([])
true = torch.tensor([])
for inputs, labels in tqdm(val_loader):
    inputs = inputs.to(device)
    labels = labels.to(device)
    logits = classifier(inputs)
    # current_loss = loss_fn(logits, labels)
    # total_val_loss += current_loss.item()
    pred = torch.cat([pred, torch.argmax(logits, dim=1).cpu()], dim=0)
    true = torch.cat([true, labels.cpu()], dim=0)

print(f"acc: {accuracy_score(true, pred)}")
print(f"f1: {f1_score(true, pred)}")
print(f"auc: {roc_auc_score(true, pred)}")

100%|██████████| 231/231 [00:01<00:00, 167.94it/s]


acc: 0.39370225768120864
f1: 0.530724862372062
auc: 0.4751307836003426


In [ ]:
test_path = '/content/drive/MyDrive/Courses/CSCI544-ANLP/Project/FNC_emb/fake-real-testsplit.tsv'
click_real_path = "/content/drive/MyDrive/Courses/CSCI544-ANLP/Project/FNC_emb/click-real-testsplit.tsv"

In [ ]:
test_df = pd.read_csv(test_path, sep="\t")
test_df

,embedding,label
0,"tensor([[-1.5196e-02, -5.1422e-02, 4.2719e-02...",0
1,"tensor([[ 2.5563e-03, -4.8680e-02, 4.7167e-02...",0
2,"tensor([[-5.8091e-02, 1.3532e-01, 6.4604e-02...",0
3,"tensor([[ 1.2593e-01, 2.0879e-02, 7.2094e-03...",1
4,"tensor([[ 2.2553e-02, 4.8011e-02, 7.3066e-02...",1
...,...,...
59995,"tensor([[ 3.2748e-02, 3.9058e-02, 1.2447e-02...",0
59996,"tensor([[ 4.0646e-02, 1.3207e-01, -1.7595e-02...",1
59997,"tensor([[ 1.0224e-01, -1.8837e-02, 1.9298e-02...",0
59998,"tensor([[-1.9212e-04, -6.1307e-02, -1.4193e-02...",1


In [ ]:
c_r_test_df = pd.read_csv(click_real_path, sep='\t')
c_r_test_df

,embedding,label
0,"tensor([[-3.2209e-02, 1.0319e-04, 2.7132e-02...",0
1,"tensor([[ 7.5430e-02, 1.0106e-01, -1.0290e-02...",0
2,"tensor([[-4.3680e-02, 3.0285e-02, -2.8098e-02...",0
3,"tensor([[ 1.2593e-01, 2.0879e-02, 7.2094e-03...",1
4,"tensor([[ 2.2553e-02, 4.8011e-02, 7.3066e-02...",1
...,...,...
59995,"tensor([[-4.8969e-02, 4.2569e-02, 2.3686e-02...",0
59996,"tensor([[ 4.0646e-02, 1.3207e-01, -1.7595e-02...",1
59997,"tensor([[-1.2044e-01, 4.8142e-02, 6.7209e-02...",0
59998,"tensor([[-1.9212e-04, -6.1307e-02, -1.4193e-02...",1


In [ ]:
test_df['embedding'] = test_df['embedding'].apply(lambda x: eval(x))

In [ ]:
c_r_test_df['embedding'] = c_r_test_df['embedding'].apply(lambda x: eval(x))

In [ ]:
test_embeddings = torch.cat(list(test_df.embedding.values), dim=0)

In [ ]:
c_r_test_embeddings = torch.cat(list(c_r_test_df.embedding.values), dim=0)

In [ ]:
test_embeddings.shape

torch.Size([60000, 768])

In [ ]:
c_r_test_embeddings.shape

torch.Size([60000, 768])

In [ ]:
y_test = test_df['label'].values
y_test

array([0, 0, 0, ..., 0, 1, 0])

In [ ]:
y_test = c_r_test_df['label'].values
y_test

array([0, 0, 0, ..., 0, 1, 0])

In [ ]:
classifier = torch.nn.Sequential(
    torch.nn.Linear(768, 256),
    torch.nn.ReLU(),
    torch.nn.Linear(256, 2)
)
classifier.load_state_dict(torch.load('liar-rb-stmt-avg-bvalf1.pt'))
classifier = classifier.to(device)
classifier

Sequential(
  (0): Linear(in_features=768, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=2, bias=True)
)

In [ ]:
test_dataset = torch.utils.data.TensorDataset(
    c_r_test_embeddings,
    torch.tensor(y_test)
)
batch_size = 256
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle = False)


In [ ]:
pred = torch.tensor([])
true = torch.tensor([])
for inputs, labels in tqdm(test_loader):
    inputs = inputs.to(device)
    labels = labels.to(device)
    logits = classifier(inputs)
    # current_loss = loss_fn(logits, labels)
    # total_val_loss += current_loss.item()
    pred = torch.cat([pred, torch.argmax(logits, dim=1).cpu()], dim=0)
    true = torch.cat([true, labels.cpu()], dim=0)

print(f"acc: {accuracy_score(true, pred)}")
print(f"f1: {f1_score(true, pred)}")
print(f"auc: {roc_auc_score(true, pred)}")

100%|██████████| 235/235 [00:01<00:00, 147.07it/s]


acc: 0.4448666666666667
f1: 0.5970676473434627
auc: 0.4436999572899229


In [ ]:
train_embeddings = torch.from_numpy(np.load('liar-rb-avg-stmt-train.npy'))
val_embeddings = torch.from_numpy(np.load('liar-rb-avg-stmt-val.npy'))
test_embeddings = torch.from_numpy(np.load('liar-rb-avg-stmt-test.npy'))


In [ ]:
train_dataset = torch.utils.data.TensorDataset(
    train_embeddings,
    torch.tensor(y_train)
)

val_dataset = torch.utils.data.TensorDataset(
    val_embeddings,
    torch.tensor(y_val)
)

test_dataset = torch.utils.data.TensorDataset(
    test_embeddings,
    torch.tensor(y_test)
)

In [ ]:
batch_size = 256
train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle = False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
classifier = torch.nn.Sequential(
    torch.nn.Linear(768, 256),
    torch.nn.ReLU(),
    torch.nn.Linear(256, 2)
)
classifier

Sequential(
  (0): Linear(in_features=768, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=2, bias=True)
)

In [ ]:
optimizer = torch.optim.Adam(classifier.parameters(), lr=1e-4)
epochs = 200
classifier = classifier.to(device)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
best_val_f1 = 0
best_test_f1 = 0
best_val_auc = 0
best_test_auc = 0
best_val_acc = 0
best_test_acc = 0

f1_val_model = {}
f1_test_model = {}

acc_val_model = {}
acc_test_model = {}

auc_val_model = {}
auc_test_model = {}

for epoch in range(epochs):

  print(f"Epoch {epoch + 1}/ {epochs}")
  print("-" * 100)
  # Train loop
  classifier.train()
  total_train_loss = 0

  for inputs, labels in tqdm(train_loader):
    inputs = inputs.to(device)
    labels = labels.to(device)
    logits = classifier(inputs)
    current_loss = loss_fn(logits, labels)
    total_train_loss += current_loss.item()
    classifier.zero_grad()
    current_loss.backward()
    optimizer.step()

  avg_train_loss = total_train_loss / len(train_loader)

  classifier.eval()
  total_val_loss = 0
  pred = torch.tensor([])
  true = torch.tensor([])
  for inputs, labels in tqdm(val_loader):
    inputs = inputs.to(device)
    labels = labels.to(device)
    logits = classifier(inputs)
    current_loss = loss_fn(logits, labels)
    total_val_loss += current_loss.item()
    pred = torch.cat([pred, torch.argmax(logits, dim=1).cpu()], dim=0)
    true = torch.cat([true, labels.cpu()], dim=0)
  avg_val_loss = total_val_loss / len(val_loader)
  val_acc = accuracy_score(true, pred)
  val_auc = roc_auc_score(true, pred)
  val_f1 = f1_score(true, pred)
  if val_f1 > best_val_f1:
    f1_val_model = get_best_model(classifier, true, pred, 'liar-rb-stmt-avg-bvalf1.pt')
    best_val_f1 = val_f1

  if val_acc > best_val_acc:
    acc_val_model = get_best_model(classifier, true, pred, 'liar-rb-stmt-avg-bvalacc.pt')
    best_val_acc = val_acc

  if val_auc > best_val_auc:
    auc_val_model = get_best_model(classifier, true, pred, 'liar-rb-stmt-avg-bvalauc.pt')
    best_val_auc = val_auc

  classifier.eval()
  total_test_loss = 0
  pred = torch.tensor([])
  true = torch.tensor([])
  for inputs, labels in tqdm(test_loader):
    inputs = inputs.to(device)
    labels = labels.to(device)
    logits = classifier(inputs)
    current_loss = loss_fn(logits, labels)
    total_test_loss += current_loss.item()
    pred = torch.cat([pred, torch.argmax(logits, dim=1).cpu()], dim=0)
    true = torch.cat([true, labels.cpu()], dim=0)
  avg_test_loss = total_test_loss / len(test_loader)
  test_acc = accuracy_score(true, pred)
  test_f1 = f1_score(true, pred)
  test_auc = roc_auc_score(true, pred)
  if test_f1 > best_test_f1:
    f1_test_model = get_best_model(classifier, true, pred, 'liar-rb-stmt-avg-btestf1.pt')
    best_test_f1 = test_f1

  if test_acc > best_test_acc:
    acc_test_model = get_best_model(classifier, true, pred, 'liar-rb-stmt-avg-btestacc.pt')
    best_test_acc = test_acc

  if test_auc > best_test_auc:
    auc_test_model = get_best_model(classifier, true, pred, 'liar-rb-stmt-avg-btestauc.pt')
    best_test_auc = test_auc

  print(f"Train loss: {avg_train_loss}, Val loss: {avg_val_loss}, Test loss: {avg_test_loss}")
  print(f"Val acc: {val_acc}, Test acc: {test_acc}")
  print(f"Val f1: {val_f1}, Test f1: {test_f1}")


Epoch 1/ 200
----------------------------------------------------------------------------------------------------


100%|██████████| 5/5 [00:00<00:00, 337.66it/s]


Train loss: 0.6807685270905495, Val loss: 0.6841397682825724, Test loss: 0.6709428906440735
Val acc: 0.5264797507788161, Test acc: 0.5659037095501184
Val f1: 0.047021943573667714, Test f1: 0.038461538461538464
Epoch 2/ 200
----------------------------------------------------------------------------------------------------


100%|██████████| 5/5 [00:00<00:00, 373.60it/s]


Train loss: 0.6666918352246285, Val loss: 0.6734551588694254, Test loss: 0.6598305821418762
Val acc: 0.5802180685358256, Test acc: 0.5895816890292028
Val f1: 0.35449101796407184, Test f1: 0.32989690721649484
Epoch 3/ 200
----------------------------------------------------------------------------------------------------


...

Train loss: 0.4437213227152824, Val loss: 0.64266503850619, Test loss: 0.6797657966613769
Val acc: 0.6347352024922118, Test acc: 0.6337805840568271
Val f1: 0.5925282363162468, Test f1: 0.5564053537284895
Epoch 199/ 200
----------------------------------------------------------------------------------------------------


100%|██████████| 5/5 [00:00<00:00, 275.08it/s]


Train loss: 0.4427484184503555, Val loss: 0.6491038898626963, Test loss: 0.6818380713462829
Val acc: 0.6347352024922118, Test acc: 0.6282557221783741
Val f1: 0.5831111111111111, Test f1: 0.533201189296333
Epoch 200/ 200
----------------------------------------------------------------------------------------------------


100%|██████████| 5/5 [00:00<00:00, 284.25it/s]

Train loss: 0.44182414188981056, Val loss: 0.6450686951478323, Test loss: 0.682219123840332
Val acc: 0.632398753894081, Test acc: 0.6322020520915549
Val f1: 0.5979557069846678, Test f1: 0.5661080074487896


In [ ]:
print("-" * 100)
print("Using AVG token Embedding")
print("-" * 100)
print("Best Validation Models")
print(f"F1 score: {f1_val_model}")
print(f"Acc score: {acc_val_model}")
print(f"AUC score: {auc_val_model}")
print("-" * 100)
print("Best Test Models")
print(f"F1 score: {f1_test_model}")
print(f"Acc score: {acc_test_model}")
print(f"AUC score: {auc_test_model}")

----------------------------------------------------------------------------------------------------
Using AVG token Embedding
----------------------------------------------------------------------------------------------------
Best Validation Models
F1 score: {'acc': 0.6456386292834891, 'auc': 0.644772144023641, 'f1': 0.6279640228945216}
Acc score: {'acc': 0.6635514018691588, 'auc': 0.6599025974025974, 'f1': 0.619047619047619}
AUC score: {'acc': 0.6635514018691588, 'auc': 0.6599025974025974, 'f1': 0.619047619047619}
----------------------------------------------------------------------------------------------------
Best Test Models
F1 score: {'acc': 0.6369376479873717, 'auc': 0.6326100060277277, 'f1': 0.5900178253119429}
Acc score: {'acc': 0.6456195737963694, 'auc': 0.6217600964436407, 'f1': 0.5166846071044134}
AUC score: {'acc': 0.6369376479873717, 'auc': 0.6326100060277277, 'f1': 0.5900178253119429}


device(type='cuda')

In [ ]:
bert_model = RobertaModel.from_pretrained('roberta-base')
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

for param in bert_model.parameters():
    param.requires_grad = False

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
bert_model.to(device)

RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(50265, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0-11): 12 x RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropou

## Types of embeddings stored:
 - Title CLS token only
 - Title avg tokens

### Trian

In [ ]:
statements_with_CLS_token = csv.writer(open("fakeddit-rb-cls-stmt-train.tsv", 'w'), delimiter='\t')
statements_with_avg_token = csv.writer(open("fakeddit-rb-avg-stmt-train.tsv", 'w'), delimiter='\t')

statements_with_CLS_token.writerow(["embedding", "label"])
statements_with_avg_token.writerow(["embedding", "label"])

17

In [ ]:
with torch.no_grad():
  for idx, statement in enumerate(tqdm(X_train_statement)):
    encoded = tokenizer(statement, truncation=True, return_tensors='pt', add_special_tokens=True)
    input_ids = encoded.input_ids.to(device)
    attention_mask = encoded.attention_mask.to(device)
    
    embedding = bert_model(input_ids, attention_mask=attention_mask)
    embedding_CLS = embedding.last_hidden_state[:, 0, :] # CLS Embedding
    embedding_avg = torch.mean(embedding.last_hidden_state, dim=1) # Mean embedding
    
    label = y_train[idx]

    statements_with_CLS_token.writerow([embedding_CLS.cpu(), label])
    statements_with_avg_token.writerow([embedding_avg.cpu(), label])

  1%|          | 3332/564000 [02:37<7:22:20, 21.12it/s]


KeyboardInterrupt: ignored

### Val

In [ ]:
statements_with_CLS_token = csv.writer(open("fakeddit-rb-cls-stmt-val.tsv", 'w'), delimiter='\t')
statements_with_avg_token = csv.writer(open("fakeddit-rb-avg-stmt-val.tsv", 'w'), delimiter='\t')

statements_with_CLS_token.writerow(["embedding", "label"])
statements_with_avg_token.writerow(["embedding", "label"])

17

In [ ]:
with torch.no_grad():
  for idx, statement in enumerate(tqdm(X_val_statement)):
    encoded = tokenizer(statement, truncation=True, return_tensors='pt', add_special_tokens=True)
    input_ids = encoded.input_ids.to(device)
    attention_mask = encoded.attention_mask.to(device)
    
    embedding = bert_model(input_ids, attention_mask=attention_mask)
    embedding_CLS = embedding.last_hidden_state[:, 0, :] # CLS Embedding
    embedding_avg = torch.mean(embedding.last_hidden_state, dim=1) # Mean embedding
    
    label = y_val[idx]

    statements_with_CLS_token.writerow([embedding_CLS.cpu(), label])
    statements_with_avg_token.writerow([embedding_avg.cpu(), label])

 33%|███▎      | 19455/59342 [08:53<18:13, 36.49it/s]


KeyboardInterrupt: ignored

### Test:

In [ ]:
statements_with_CLS_token = csv.writer(open("fakeddit-rb-cls-stmt-test.tsv", 'w'), delimiter='\t')
statements_with_avg_token = csv.writer(open("fakeddit-rb-avg-stmt-test.tsv", 'w'), delimiter='\t')

statements_with_CLS_token.writerow(["embedding", "label"])
statements_with_avg_token.writerow(["embedding", "label"])

17

In [ ]:
with torch.no_grad():
  for idx, statement in enumerate(tqdm(X_test_statement)):
    encoded = tokenizer(statement, truncation=True, return_tensors='pt', add_special_tokens=True)
    input_ids = encoded.input_ids.to(device)
    attention_mask = encoded.attention_mask.to(device)
    
    embedding = bert_model(input_ids, attention_mask=attention_mask)
    embedding_CLS = embedding.last_hidden_state[:, 0, :] # CLS Embedding
    embedding_avg = torch.mean(embedding.last_hidden_state, dim=1) # Mean embedding
    
    label = y_test[idx]

    statements_with_CLS_token.writerow([embedding_CLS.cpu(), label])
    statements_with_avg_token.writerow([embedding_avg.cpu(), label])

  0%|          | 1018/564000 [00:44<5:35:07, 28.00it/s]

In [ ]:
statements_with_CLS_token = []
statements_with_avg_token = []

In [ ]:
with torch.no_grad():
  for statement in tqdm(X_test_statement):
    encoded = tokenizer(statement, truncation=True, return_tensors='pt', add_special_tokens=True)
    input_ids = encoded.input_ids.to(device)
    attention_mask = encoded.attention_mask.to(device)
    
    embedding = bert_model(input_ids, attention_mask=attention_mask)
    embedding_CLS = embedding.last_hidden_state[:, 0, :] # CLS Embedding
    embedding_avg = torch.mean(embedding.last_hidden_state, dim=1) # Mean embedding
    
    statements_with_CLS_token.append(embedding_CLS.cpu())
    statements_with_avg_token.append(embedding_avg.cpu())

100%|██████████| 1267/1267 [00:20<00:00, 61.07it/s]


In [ ]:
statements_with_CLS_token = torch.cat(statements_with_CLS_token, dim=0)
statements_with_avg_token = torch.cat(statements_with_avg_token, dim=0)

In [ ]:
np.save('liar-rb-cls-stmt-test.npy', statements_with_CLS_token.numpy())
np.save('liar-rb-avg-stmt-test.npy', statements_with_avg_token.numpy())

In [ ]:
del statements_with_CLS_token
del statements_with_avg_token

In [10]:
y_test.shape

(1267,)

In [16]:
train_embeddings = torch.from_numpy(np.load('liar-rb-avg-stmt-train.npy'))
train_embeddings.shape

torch.Size([10240, 768])

In [17]:
val_embeddings = torch.from_numpy(np.load('liar-rb-avg-stmt-val.npy'))
val_embeddings.shape

torch.Size([1284, 768])

In [18]:
test_embeddings = torch.from_numpy(np.load('liar-rb-avg-stmt-test.npy'))
test_embeddings.shape

torch.Size([1267, 768])

In [19]:
class Mod(torch.nn.Module):
  def __init__(self, num_hidden, num_outs):
    super().__init__()
    
    self.fc1 = torch.nn.Linear(768,num_hidden)
    
    #self.fc2 = torch.nn.Linear(500,num_hidden)
    self.drop = torch.nn.Dropout(0.33)
    self.hidden = torch.nn.Linear(num_hidden,num_outs)

  def forward(self,x):
   # x = self.pool(x)
    x = torch.nn.functional.relu(self.fc1(x))
    #x = torch.nn.Dropout(0.2)(x)
    x = self.drop(x)
   
    #x = torch.nn.functional.relu(self.fc2(x))
    #x = torch.nn.Dropout(0.2)(x)
    return self.hidden(x)

In [ ]:
! unzip "/content/drive/MyDrive/Courses/CSCI544-ANLP/Project/544 Project Michael/Models.zip" -d /content/drive/MyDrive/Courses/CSCI544-ANLP/Project/michael_models/

Archive:  /content/drive/MyDrive/Courses/CSCI544-ANLP/Project/544 Project Michael/Models.zip
  inflating: /content/drive/MyDrive/Courses/CSCI544-ANLP/Project/michael_models/model_trained_FNC_Click.pt  
  inflating: /content/drive/MyDrive/Courses/CSCI544-ANLP/Project/michael_models/model_trained_FNC_Fake.pt  
  inflating: /content/drive/MyDrive/Courses/CSCI544-ANLP/Project/michael_models/model_trained_on_fakeddit.pt  


In [20]:
classifier = Mod(300, 2)


In [32]:
# classifier.load_state_dict(torch.load('/content/drive/MyDrive/Courses/CSCI544-ANLP/Project/michael_models/model_trained_on_fakeddit.pt'))
classifier.load_state_dict(torch.load('/content/drive/MyDrive/Courses/CSCI544-ANLP/CSCI544-Project/Shashank/michael_models/combined_model.pt'))

<All keys matched successfully>

In [35]:
for param in classifier.parameters():
  param.requires_grad = False

In [36]:
for name, param in classifier.named_parameters():
  print(name, param.requires_grad)

fc1.weight False
fc1.bias False
hidden.weight False
hidden.bias False


In [37]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [38]:
train_dataset = torch.utils.data.TensorDataset(
    train_embeddings,
    torch.tensor(y_train)
)

val_dataset = torch.utils.data.TensorDataset(
    val_embeddings,
    torch.tensor(y_val)
)

test_dataset = torch.utils.data.TensorDataset(
    test_embeddings,
    torch.tensor(y_test)
)

In [39]:
batch_size = 256
train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle = False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [40]:
optimizer = torch.optim.Adam(classifier.parameters(), lr=1e-4)
epochs = 300
classifier = classifier.to(device)
loss_fn = nn.CrossEntropyLoss()

In [41]:
def get_best_model(model, true, pred, model_name):
  torch.save(model.state_dict(), model_name)
  return {
      "acc": accuracy_score(true, pred),
      "auc": roc_auc_score(true, pred),
      "f1": f1_score(true, pred),
  }

In [44]:
best_val_f1 = 0
best_test_f1 = 0
best_val_auc = 0
best_test_auc = 0
best_val_acc = 0
best_test_acc = 0

f1_val_model = {}
f1_test_model = {}

acc_val_model = {}
acc_test_model = {}

auc_val_model = {}
auc_test_model = {}

for epoch in range(epochs):

#   print(f"Epoch {epoch + 1}/ {epochs}")
#   print("-" * 100)
#   # Train loop
#   classifier.train()
#   total_train_loss = 0

#   for inputs, labels in tqdm(train_loader):
#     inputs = inputs.to(device)
#     labels = labels.to(device)
#     logits = classifier(inputs)
#     current_loss = loss_fn(logits, labels)
#     total_train_loss += current_loss.item()
#     classifier.zero_grad()
#     current_loss.backward()
#     optimizer.step()

#   avg_train_loss = total_train_loss / len(train_loader)

  classifier.eval()
  total_val_loss = 0
  pred = torch.tensor([])
  true = torch.tensor([])
  for inputs, labels in tqdm(val_loader):
    inputs = inputs.to(device)
    labels = labels.to(device)
    logits = classifier(inputs)
    current_loss = loss_fn(logits, labels)
    total_val_loss += current_loss.item()
    pred = torch.cat([pred, torch.argmax(logits, dim=1).cpu()], dim=0)
    true = torch.cat([true, labels.cpu()], dim=0)
  avg_val_loss = total_val_loss / len(val_loader)
  val_acc = accuracy_score(true, pred)
  val_auc = roc_auc_score(true, pred)
  val_f1 = f1_score(true, pred)
  if val_f1 > best_val_f1:
    f1_val_model = get_best_model(classifier, true, pred, 'fakeddit-finetuned-liar-rb-stmt-avg-bvalf1.pt')
    best_val_f1 = val_f1

  if val_acc > best_val_acc:
    acc_val_model = get_best_model(classifier, true, pred, 'fakeddit-finetuned-liar-rb-stmt-avg-bvalacc.pt')
    best_val_acc = val_acc

  if val_auc > best_val_auc:
    auc_val_model = get_best_model(classifier, true, pred, 'fakeddit-finetuned-liar-rb-stmt-avg-bvalauc.pt')
    best_val_auc = val_auc

  classifier.eval()
  total_test_loss = 0
  pred = torch.tensor([])
  true = torch.tensor([])
  for inputs, labels in tqdm(test_loader):
    inputs = inputs.to(device)
    labels = labels.to(device)
    logits = classifier(inputs)
    current_loss = loss_fn(logits, labels)
    total_test_loss += current_loss.item()
    pred = torch.cat([pred, torch.argmax(logits, dim=1).cpu()], dim=0)
    true = torch.cat([true, labels.cpu()], dim=0)
  avg_test_loss = total_test_loss / len(test_loader)
  test_acc = accuracy_score(true, pred)
  test_f1 = f1_score(true, pred)
  test_auc = roc_auc_score(true, pred)
  if test_f1 > best_test_f1:
    f1_test_model = get_best_model(classifier, true, pred, 'fakeddit-finetuned-liar-rb-stmt-avg-btestf1.pt')
    best_test_f1 = test_f1

  if test_acc > best_test_acc:
    acc_test_model = get_best_model(classifier, true, pred, 'fakeddit-finetuned-liar-rb-stmt-avg-btestacc.pt')
    best_test_acc = test_acc

  if test_auc > best_test_auc:
    auc_test_model = get_best_model(classifier, true, pred, 'fakeddit-finetuned-liar-rb-stmt-avg-btestauc.pt')
    best_test_auc = test_auc

  print(f"Train loss: {avg_train_loss}, Val loss: {avg_val_loss}, Test loss: {avg_test_loss}")
  print(f"Val acc: {val_acc}, Test acc: {test_acc}")
  print(f"Val f1: {val_f1}, Test f1: {test_f1}")


100%|██████████| 5/5 [00:00<00:00, 81.23it/s]


Train loss: 0.40951759517192843, Val loss: 0.8896880149841309, Test loss: 0.9382977962493897
Val acc: 0.5163551401869159, Test acc: 0.47829518547750594
Val f1: 0.3695431472081218, Test f1: 0.3601161665053243


100%|██████████| 5/5 [00:00<00:00, 143.65it/s]


Train loss: 0.40951759517192843, Val loss: 0.8896880149841309, Test loss: 0.9382977962493897
Val acc: 0.5163551401869159, Test acc: 0.47829518547750594
Val f1: 0.3695431472081218, Test f1: 0.3601161665053243


100%|██████████| 5/5 [00:00<00:00, 238.51it/s]


...

Train loss: 0.40951759517192843, Val loss: 0.8896880149841309, Test loss: 0.9382977962493897
Val acc: 0.5163551401869159, Test acc: 0.47829518547750594
Val f1: 0.3695431472081218, Test f1: 0.3601161665053243


100%|██████████| 5/5 [00:00<00:00, 292.14it/s]


Train loss: 0.40951759517192843, Val loss: 0.8896880149841309, Test loss: 0.9382977962493897
Val acc: 0.5163551401869159, Test acc: 0.47829518547750594
Val f1: 0.3695431472081218, Test f1: 0.3601161665053243


100%|██████████| 5/5 [00:00<00:00, 283.25it/s]


Train loss: 0.40951759517192843, Val loss: 0.8896880149841309, Test loss: 0.9382977962493897
Val acc: 0.5163551401869159, Test acc: 0.47829518547750594
Val f1: 0.3695431472081218, Test f1: 0.3601161665053243


In [45]:
print("-" * 100)
print("Finetuning Fakeddit model on LIAR")
print("-" * 100)
print("Best Validation Models")
print(f"F1 score: {f1_val_model}")
print(f"Acc score: {acc_val_model}")
print(f"AUC score: {auc_val_model}")
print("-" * 100)
print("Best Test Models")
print(f"F1 score: {f1_test_model}")
print(f"Acc score: {acc_test_model}")
print(f"AUC score: {auc_test_model}")

----------------------------------------------------------------------------------------------------
Finetuning Fakeddit model on LIAR
----------------------------------------------------------------------------------------------------
Best Validation Models
F1 score: {'acc': 0.5163551401869159, 'auc': 0.5266496228322576, 'f1': 0.3695431472081218}
Acc score: {'acc': 0.5163551401869159, 'auc': 0.5266496228322576, 'f1': 0.3695431472081218}
AUC score: {'acc': 0.5163551401869159, 'auc': 0.5266496228322576, 'f1': 0.3695431472081218}
----------------------------------------------------------------------------------------------------
Best Test Models
F1 score: {'acc': 0.47829518547750594, 'auc': 0.5099989362833741, 'f1': 0.3601161665053243}
Acc score: {'acc': 0.47829518547750594, 'auc': 0.5099989362833741, 'f1': 0.3601161665053243}
AUC score: {'acc': 0.47829518547750594, 'auc': 0.5099989362833741, 'f1': 0.3601161665053243}


In [ ]:
pred = torch.tensor([])
true = torch.tensor([])
for inputs, labels in tqdm(test_loader):
    inputs = inputs.to(device)
    labels = labels.to(device)
    logits = model_FNC(inputs)
    # current_loss = loss_fn(logits, labels)
    # total_val_loss += current_loss.item()
    pred = torch.cat([pred, torch.argmax(logits, dim=1).cpu()], dim=0)
    true = torch.cat([true, labels.cpu()], dim=0)

print(f"acc: {accuracy_score(true, pred)}")
print(f"f1: {f1_score(true, pred)}")
print(f"auc: {roc_auc_score(true, pred)}")

100%|██████████| 5/5 [00:00<00:00, 129.31it/s]

acc: 0.5185477505919495
f1: 0.5617816091954023
auc: 0.514315852923448
